In [1]:
import json
import pickle
import numpy as np
from queue import Queue
from tqdm import tqdm
from rdkit import Chem
from collections import Counter
from tqdm import trange, tqdm
from BLEU_utils import *

In [2]:
all_routes = pickle.load(open('data/all_routes.pickle', 'rb'))
all_templates = json.load(open('data/all_routes_templates_1_0_0.json'))
golden_dict = pickle.load(open('data/golden_dict.pickle', 'rb'))
n5_routes = json.load(open('data/n5-routes.json'))
n1_routes = json.load(open('data/n1-routes.json'))
n5_mcts_top10_routes = json.load(open('data/n5_mcts_firstroutes_10.json'))
n1_mcts_top10_routes = json.load(open('data/n1_mcts_firstroutes_10.json'))
n5_retrostar_top10_routes = json.load(open('data/n5_retrostar_firstroutes_10.json'))
n1_retrostar_top10_routes = json.load(open('data/n1_retrostar_firstroutes_10.json'))
n5_mcts_top1_routes = json.load(open('data/n5_mcts_firstroutes.json'))
n1_mcts_top1_routes = json.load(open('data/n1_mcts_firstroutes.json'))
n5_retrostar_top1_routes = json.load(open('data/n5_retrostar_firstroutes.json'))
n1_retrostar_top1_routes = json.load(open('data/n1_retrostar_firstroutes.json'))
np.random.seed(0)

In [3]:
all_ngram_rxns, all_ngram_templates, all_rxns = build_vocab(all_routes, all_templates)
set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in all_ngram_rxns]
set_ngram_templates = [set(ngram_templates) for ngram_templates in all_ngram_templates]
print('Number of n-gram rxns for n in [2, 3, 4, 5]:', [len(ngram_rxns) for ngram_rxns in all_ngram_rxns])
print('Number of n-gram rxns for n in [2, 3, 4, 5] (after removing duplicates):', [len(ngram_rxns) for ngram_rxns in set_ngram_rxns])
print('Number of n-gram templates for n in [2, 3, 4, 5] (after removing duplicates):', [len(ngram_templates) for ngram_templates in set_ngram_templates])

Number of n-gram rxns for n in [2, 3, 4, 5]: [818622, 342358, 141804, 56046]
Number of n-gram rxns for n in [2, 3, 4, 5] (after removing duplicates): [253758, 106978, 44796, 17836]
Number of n-gram templates for n in [2, 3, 4, 5] (after removing duplicates): [112048, 69457, 31191, 12816]


In [4]:
vocab_routes = []
test_routes = []
for idx, route in enumerate(all_routes):
    rxn_nodes = extract_rxns(route)
    ID = rxn_nodes[0]['metadata']['ID'].split(';')[0][2:]
    if ID[:4] in ['2014', '2015', '2016']:
        test_routes.append(route)
    else:
        vocab_routes.append(route)
print('Number of vocab routes:', len(vocab_routes))
print('Number of test routes:', len(test_routes))
print('Average length of test routes: {:.4f}'.format(np.mean([len(extract_rxns(route)) for route in test_routes])))
all_ngram_rxns, all_ngram_templates, all_rxns = build_vocab(vocab_routes, all_templates)
all_ngram_templates = [[tuple([all_templates[rxn] for rxn in curr_rxn_set]) for curr_rxn_set in n_gram_rxns] for n_gram_rxns in all_ngram_rxns]
set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in all_ngram_rxns]
set_ngram_templates = [set(ngram_templates) for ngram_templates in all_ngram_templates]
print('Number of n-gram rxns for n in [2, 3, 4, 5] in routes before 2013:', [len(ngram_rxns) for ngram_rxns in all_ngram_rxns])
print('Number of n-gram rxns for n in [2, 3, 4, 5] in routes before 2013 (after removing duplicates):', [len(ngram_rxns) for ngram_rxns in set_ngram_rxns])
print('Number of n-gram templates for n in [2, 3, 4, 5] in routes before 2013 (after removing duplicates):', [len(ngram_templates) for ngram_templates in set_ngram_templates])

Number of vocab routes: 395121
Number of test routes: 62326
Average length of test routes: 2.8735
Number of n-gram rxns for n in [2, 3, 4, 5] in routes before 2013: [701854, 290944, 119587, 46628]
Number of n-gram rxns for n in [2, 3, 4, 5] in routes before 2013 (after removing duplicates): [227607, 95849, 39985, 15895]
Number of n-gram templates for n in [2, 3, 4, 5] in routes before 2013 (after removing duplicates): [102481, 62377, 27888, 11426]


In [5]:
bleu_ratio, bleu_template_ratio = evaluate_routes(test_routes, all_templates, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5] in routes after 2014:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5] in patents after 2014:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5] in patents after 2014:', [len(bleu_ratio[i]) / len(test_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5] in routes after 2014: [0.6996204799240451, 0.7093509749685522, 0.7057417117264733, 0.7172911202697639]
n-gram template ratio for n in [2, 3, 4, 5] in patents after 2014: [0.8236269022598164, 0.7340199872266381, 0.718275165219291, 0.7224085175471462]
n-gram existence ratio for n in [2, 3, 4, 5] in patents after 2014: [1.0, 0.445817154959407, 0.19584122196194204, 0.08564643968809164]


In [6]:
vocab_routes = []
test_routes = []
ID_random_dict = {}
for idx, route in enumerate(all_routes):
    rxn_nodes = extract_rxns(route)
    ID = rxn_nodes[0]['metadata']['ID'].split(';')[0][2:]
    if ID not in ID_random_dict:
        ID_random_dict[ID] = np.random.random()
    if ID_random_dict[ID] < 0.2:
        test_routes.append(route)
    else:
        vocab_routes.append(route)
print('Number of vocab routes:', len(vocab_routes))
print('Number of test routes:', len(test_routes))
print('Average length of test routes: {:.4f}'.format(np.mean([len(extract_rxns(route)) for route in test_routes])))
all_ngram_rxns, all_ngram_templates, all_rxns = build_vocab(vocab_routes, all_templates)
all_ngram_templates = [[tuple([all_templates[rxn] for rxn in curr_rxn_set]) for curr_rxn_set in n_gram_rxns] for n_gram_rxns in all_ngram_rxns]
set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in all_ngram_rxns]
set_ngram_templates = [set(ngram_templates) for ngram_templates in all_ngram_templates]
print('Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab patents:', [len(ngram_rxns) for ngram_rxns in all_ngram_rxns])
print('Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab patents (after removing duplicates):', [len(ngram_rxns) for ngram_rxns in set_ngram_rxns])
print('Number of n-gram templates for n in [2, 3, 4, 5] in randomly 80% vocab patents (after removing duplicates):', [len(ngram_templates) for ngram_templates in set_ngram_templates])

Number of vocab routes: 364438
Number of test routes: 93009
Average length of test routes: 2.7974
Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab patents: [651443, 271861, 112181, 44505]
Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab patents (after removing duplicates): [231334, 97537, 40709, 16234]
Number of n-gram templates for n in [2, 3, 4, 5] in randomly 80% vocab patents (after removing duplicates): [103641, 63755, 28557, 11731]


In [7]:
bleu_ratio, bleu_template_ratio = evaluate_routes(test_routes, all_templates, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5] in randomly 20% test patents:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5] in randomly 20% test patents:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5] in randomly 20% test patents:', [len(bleu_ratio[i]) / len(test_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5] in randomly 20% test patents: [0.8014285088269864, 0.7888602004056551, 0.7676607273708975, 0.7843653325599381]
n-gram template ratio for n in [2, 3, 4, 5] in randomly 20% test patents: [0.892716435516326, 0.8204282384282384, 0.7886287462649836, 0.7934696442382057]
n-gram existence ratio for n in [2, 3, 4, 5] in randomly 20% test patents: [1.0, 0.422808545409584, 0.18585298196948682, 0.07414336247029857]


In [8]:
vocab_routes = []
test_routes = []
ID_random_dict = {}
for idx, route in enumerate(all_routes):
    rxn_nodes = extract_rxns(route)
    ID = rxn_nodes[0]['metadata']['ID'].split(';')[0][2:]
    if np.random.random() < 0.2:
        test_routes.append(route)
    else:
        vocab_routes.append(route)
print('Number of vocab routes:', len(vocab_routes))
print('Number of test routes:', len(test_routes))
print('Average length of test routes: {:.2f}'.format(np.mean([len(extract_rxns(route)) for route in test_routes])))
all_ngram_rxns, all_ngram_templates, all_rxns = build_vocab(vocab_routes, all_templates)
all_ngram_templates = [[tuple([all_templates[rxn] for rxn in curr_rxn_set]) for curr_rxn_set in n_gram_rxns] for n_gram_rxns in all_ngram_rxns]
set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in all_ngram_rxns]
set_ngram_templates = [set(ngram_templates) for ngram_templates in all_ngram_templates]
print('Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab routes:', [len(ngram_rxns) for ngram_rxns in all_ngram_rxns])
print('Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab routes (after removing duplicates):', [len(ngram_rxns) for ngram_rxns in set_ngram_rxns])
print('Number of n-gram templates for n in [2, 3, 4, 5] in randomly 80% vocab routes (after removing duplicates):', [len(ngram_templates) for ngram_templates in set_ngram_templates])

Number of vocab routes: 365979
Number of test routes: 91468
Average length of test routes: 2.79
Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab routes: [654801, 273711, 113266, 44725]
Number of n-gram rxns for n in [2, 3, 4, 5] in randomly 80% vocab routes (after removing duplicates): [233742, 98672, 41427, 16545]
Number of n-gram templates for n in [2, 3, 4, 5] in randomly 80% vocab routes (after removing duplicates): [105449, 64823, 29157, 12008]


In [9]:
bleu_ratio, bleu_template_ratio = evaluate_routes(test_routes, all_templates, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5] in randomly 20% test routes:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5] in randomly 20% test routes:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5] in randomly 20% test routes:', [len(bleu_ratio[i]) / len(test_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5] in randomly 20% test routes: [0.8325197518986605, 0.8314357341463349, 0.8312066310384697, 0.838586517879075]
n-gram template ratio for n in [2, 3, 4, 5] in randomly 20% test routes: [0.9419958910081141, 0.9049685483634883, 0.8993247090780723, 0.9013678632838543]
n-gram existence ratio for n in [2, 3, 4, 5] in randomly 20% test routes: [1.0, 0.42087943324441335, 0.18041282197052522, 0.07417894782874886]


In [10]:
np.mean([len(extract_rxns(route)) for route in all_routes])

2.789545018329992

In [3]:
no_mapping_dict = {}
def rxn2mols(rxn):
    r, p = rxn.split('>')[0], rxn.split('>')[-1]
    rs = (r+'.'+p).split('.')
    r_mols = [Chem.MolFromSmiles(r) for r in rs]
    for mol in r_mols:
        for atom in mol.GetAtoms():
            atom.SetAtomMapNum(0)
    r_smls = [Chem.MolToSmiles(r_mol) for r_mol in r_mols]
    return tuple(sorted(r_smls))

import os
if not os.path.exists('data/no_mapping_dict.pkl'):
    all_ngram_rxns, all_ngram_templates, all_rxns = build_vocab(all_routes, all_templates)
    for rxn in tqdm(set(all_rxns)):
        no_mapping_dict[rxn] = rxn2mols(rxn)
    pickle.dump(no_mapping_dict, open('data/no_mapping_dict.pkl', 'wb'))
else:
    no_mapping_dict = pickle.load(open('data/no_mapping_dict.pkl', 'rb'))

In [4]:
n1_patents = set([extract_rxns(route)[0]['metadata']['ID'].split(';')[0] for route in n1_routes])
n5_patents = set([extract_rxns(route)[0]['metadata']['ID'].split(';')[0] for route in n5_routes])
n1_known_routes = [route for route in all_routes if extract_rxns(route)[0]['metadata']['ID'].split(';')[0] not in n1_patents]
n5_known_routes = [route for route in all_routes if extract_rxns(route)[0]['metadata']['ID'].split(';')[0] not in n5_patents]

In [5]:
n1_ngram_rxns, n1_ngram_templates, n1_rxns = build_vocab(n1_known_routes, all_templates)
n1_set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in n1_ngram_rxns]
n1_set_ngram_templates = [set(ngram_templates) for ngram_templates in n1_ngram_templates]
n1_sorted_ngram_rxns = [[tuple([no_mapping_dict[rxn] for rxn in ngram]) for ngram in ngram_rxns] for ngram_rxns in n1_ngram_rxns]
n1_set_sorted_ngram_rxns = [set(ngram_rxns) for ngram_rxns in n1_sorted_ngram_rxns]

In [6]:
bleu_ratio, bleu_template_ratio = evaluate_routes(n1_routes, all_templates, n1_set_ngram_templates, n1_set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n1_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.578660873015873, 0.5767130549368223, 0.5785918621179815, 0.583462962962963]
n-gram template ratio for n in [2, 3, 4, 5]: [0.7511015873015873, 0.6210248897985939, 0.5983786425017769, 0.5937407407407408]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.7119, 0.268, 0.09]


In [7]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n1_mcts_top1_routes, n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n1_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.142138279409627, 0.10951645007466672, 0.045199501246882795, 0.013703703703703702]
n-gram template ratio for n in [2, 3, 4, 5]: [0.27003562308608436, 0.07225220513308195, 0.01974335869010946, 0.0033333333333333335]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9013, 0.4071, 0.1604, 0.09]


In [10]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes([i for j in n1_mcts_top10_routes for i in j], n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len([i for j in n1_mcts_top10_routes for i in j]) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.04518914066179242, 0.02471396538109152, 0.009203781481967594, 0.0029469044690446904]
n-gram template ratio for n in [2, 3, 4, 5]: [0.20943695459583322, 0.029861807143287802, 0.0046693224374987465, 0.001175393896796111]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9794685245699488, 0.6484061902706701, 0.3131717943975173, 0.1670879832295456]


In [11]:
n1_mcts_all = []
for i in trange(10000):
    curr_route = json.load(open('../routes/n1routes/mcts_v2/routes_{}.json'.format(i)))
    n1_mcts_all += curr_route

100%|██████████| 10000/10000 [05:07<00:00, 32.48it/s] 


In [12]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n1_mcts_all, n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n1_mcts_all) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.01276801508318214, 0.002478380844726092, 0.0005659740915677712, 0.00013437779799921107]
n-gram template ratio for n in [2, 3, 4, 5]: [0.28985220442504506, 0.03525869271452362, 0.003237809972216429, 0.0004255334090538764]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9992406862345634, 0.9753387043531226, 0.8999235506740938, 0.7905133085362198]


In [13]:
print(np.mean([len(extract_rxns(route)) for route in n1_routes]))
print(np.mean([len(extract_rxns(route)) for route in n1_mcts_top1_routes]))
print(np.mean([len(extract_rxns(route)) for route in [i for j in n1_mcts_top10_routes for i in j]]))
print(np.mean([len(extract_rxns(route)) for route in n1_mcts_all]))

3.1785
3.1781
3.9691000267176357
8.238583705451175


In [6]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n1_retrostar_top1_routes, n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n1_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.06842321232646319, 0.051267012945317776, 0.02874414014310387, 0.02882882882882883]
n-gram template ratio for n in [2, 3, 4, 5]: [0.15245536789972605, 0.02547766196935472, 0.009770540340488526, 0.007276507276507277]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9013, 0.4153, 0.1351, 0.0481]


In [7]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes([i for j in n1_retrostar_top10_routes for i in j], n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len([i for j in n1_retrostar_top10_routes for i in j]) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.03561006797176607, 0.018762452356202358, 0.00976031513403277, 0.00556367314825222]
n-gram template ratio for n in [2, 3, 4, 5]: [0.1537302666324725, 0.016662490099990098, 0.0038165608207132388, 0.0017065677569218088]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9796295531510696, 0.6356653175848682, 0.25361361196002313, 0.09185801602378789]


In [8]:
n1_retrostar_all = []
for i in trange(10000):
    curr_route = json.load(open('../routes/n1routes/retrostar_v2/routes_{}.json'.format(i)))
    n1_retrostar_all += curr_route

100%|██████████| 10000/10000 [03:59<00:00, 41.79it/s] 


In [9]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n1_retrostar_all, n1_set_ngram_templates, n1_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n1_retrostar_all) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.017078943578664667, 0.003386616656949492, 0.0007838407313628039, 0.0002141673183808219]
n-gram template ratio for n in [2, 3, 4, 5]: [0.24629085232790832, 0.02241742939826518, 0.0015240257966706412, 0.0002698683059840503]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9993387958305243, 0.9699592573237753, 0.8007342058967843, 0.508811069136226]


In [10]:
print(np.mean([len(extract_rxns(route)) for route in n1_routes]))
print(np.mean([len(extract_rxns(route)) for route in n1_retrostar_top1_routes]))
print(np.mean([len(extract_rxns(route)) for route in [i for j in n1_retrostar_top10_routes for i in j]]))
print(np.mean([len(extract_rxns(route)) for route in n1_retrostar_all]))

3.1785
2.7816
3.3811328157264393
5.042214678267159


In [11]:
n5_ngram_rxns, n5_ngram_templates, n5_rxns = build_vocab(n5_known_routes, all_templates)
n5_set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in n5_ngram_rxns]
n5_set_ngram_templates = [set(ngram_templates) for ngram_templates in n5_ngram_templates]
n5_sorted_ngram_rxns = [[tuple([no_mapping_dict[rxn] for rxn in ngram]) for ngram in ngram_rxns] for ngram_rxns in n5_ngram_rxns]
n5_set_sorted_ngram_rxns = [set(ngram_rxns) for ngram_rxns in n5_sorted_ngram_rxns]

In [12]:
bleu_ratio, bleu_template_ratio = evaluate_routes(n5_routes, all_templates, n5_set_ngram_templates, n5_set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n5_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.4895048412698413, 0.47625684866563134, 0.46442399606500173, 0.4593916755602988]
n-gram template ratio for n in [2, 3, 4, 5]: [0.6997671031746033, 0.5232870007329161, 0.48715603567484284, 0.4693970117395944]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.9291, 0.5131, 0.1874]


In [13]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n5_mcts_top1_routes, n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n5_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.0793926320863627, 0.07294642774219827, 0.034124663978494625, 0.011606677099634845]
n-gram template ratio for n in [2, 3, 4, 5]: [0.23974434080311355, 0.05509204378779358, 0.015705793982003658, 0.0027200238467844098]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9432, 0.5667, 0.248, 0.1278]


In [14]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes([i for j in n5_mcts_top10_routes for i in j], n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len([i for j in n5_mcts_top10_routes for i in j]) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.038502158991249015, 0.022369986147370152, 0.010151466501578103, 0.004193284572416134]
n-gram template ratio for n in [2, 3, 4, 5]: [0.21165676183113355, 0.029152924915662962, 0.005303162494596589, 0.001136169275693394]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9897152142835104, 0.7670393187357941, 0.41182338965967646, 0.22217194104760826]


In [15]:
n5_mcts_all = []
for i in trange(10000):
    curr_route = json.load(open('../routes/n5routes/mcts_v2/routes_{}.json'.format(i)))
    n5_mcts_all += curr_route

100%|██████████| 10000/10000 [14:06<00:00, 11.82it/s]  


In [16]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n5_mcts_all, n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n5_mcts_all) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.01448911980263224, 0.0031385344013280525, 0.0008258016991246231, 0.00016363965690208433]
n-gram template ratio for n in [2, 3, 4, 5]: [0.29075665283300234, 0.03403250450108822, 0.0030031130801968144, 0.0003126619045240292]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9996403960915381, 0.9857598204143488, 0.9263291910162562, 0.8267033615867996]


In [17]:
print(np.mean([len(extract_rxns(route)) for route in n5_routes]))
print(np.mean([len(extract_rxns(route)) for route in n5_mcts_top1_routes]))
print(np.mean([len(extract_rxns(route)) for route in [i for j in n5_mcts_top10_routes for i in j]]))
print(np.mean([len(extract_rxns(route)) for route in n5_mcts_all]))

3.8483
3.7535
4.501362734107435
8.654789711137346


In [18]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n5_retrostar_top1_routes, n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n5_routes) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.04491607428948396, 0.03443102362726579, 0.02415329768270945, 0.013798348544111255]
n-gram template ratio for n in [2, 3, 4, 5]: [0.14883901645192996, 0.020666401564105114, 0.007182115270350565, 0.0017383746197305518]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9432, 0.5748, 0.2244, 0.0767]


In [19]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes([i for j in n5_retrostar_top10_routes for i in j], n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len([i for j in n5_retrostar_top10_routes for i in j]) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.03118522206837509, 0.016561058588016875, 0.010208119845801005, 0.00613202921998333]
n-gram template ratio for n in [2, 3, 4, 5]: [0.1608672671059326, 0.016479911825791786, 0.004201434636217245, 0.0009082217973231358]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9897768653867977, 0.7504049021529446, 0.35234224290002786, 0.14027667454119686]


In [20]:
n5_retrostar_all = []
for i in trange(10000):
    curr_route = json.load(open('../routes/n5routes/retrostar_v2/routes_{}.json'.format(i)))
    n5_retrostar_all += curr_route

100%|██████████| 10000/10000 [10:10<00:00, 16.39it/s]  


In [21]:
bleu_ratio, bleu_template_ratio = evaluate_aizynthfinder_routes(n5_retrostar_all, n5_set_ngram_templates, n5_set_sorted_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(n5_retrostar_all) for i in range(4)])

n-gram rxn ratio for n in [2, 3, 4, 5]: [0.02320710795291471, 0.005656853583710317, 0.0014400652930381804, 0.00039565913494990203]
n-gram template ratio for n in [2, 3, 4, 5]: [0.2511603197851982, 0.024330319674731878, 0.001799758519190014, 0.00024391808322041036]
n-gram existence ratio for n in [2, 3, 4, 5]: [0.9996448760871204, 0.9804488923456506, 0.8463224489402195, 0.5841041677756846]


In [22]:
print(np.mean([len(extract_rxns(route)) for route in n5_routes]))
print(np.mean([len(extract_rxns(route)) for route in n5_retrostar_top1_routes]))
print(np.mean([len(extract_rxns(route)) for route in [i for j in n5_retrostar_top10_routes for i in j]]))
print(np.mean([len(extract_rxns(route)) for route in n5_retrostar_all]))

3.8483
3.2253
3.7857990241084414
5.376251642090829


In [34]:
train_routes = pickle.load(open('data/routes_train.pkl', 'rb'))
train_templates = json.load(open('data/train_routes_templates_1_0_0.json'))
test_dict = json.load(open('data/test_routes_templates_1_0_0.json'))

In [35]:
train_ngram_rxns, train_ngram_templates, train_rxns = build_train_vocab(train_routes, train_templates)
set_ngram_rxns = [set(ngram_rxns) for ngram_rxns in train_ngram_rxns]
set_ngram_templates = [set(ngram_templates) for ngram_templates in train_ngram_templates]
test_dict = json.load(open('data/test_routes_templates_1_0_0.json'))

In [36]:
golden_routes = pickle.load(open('test_routes/routes_golden.pkl', 'rb'))
print(len(golden_routes))
print(np.mean([len(route) for route in golden_routes]))
bleu_ratio, bleu_template_ratio = evaluate_routes(golden_routes, test_dict, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(golden_routes) for i in range(4)])

190
6.673684210526316
n-gram rxn ratio for n in [2, 3, 4, 5]: [0.10084247623721307, 0.04858771719236835, 0.014854627354627356, 0.0028688524590163933]
n-gram template ratio for n in [2, 3, 4, 5]: [0.42897579029157973, 0.2885029537645817, 0.21535032160032155, 0.11829625292740045]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.9052631578947369, 0.7789473684210526, 0.6421052631578947]


In [37]:
retro_routes = json.load(open('test_routes/routes_retrostar.json'))['routes']
retro_routes = [[rxn.split('>')[0] + '>>' + rxn.split('>')[-1] for rxn in route.split('|')] for route in retro_routes if isinstance(route, str)]
print(len(retro_routes))
print(np.mean([len(route) for route in retro_routes]))
bleu_ratio, bleu_template_ratio = evaluate_routes(retro_routes, test_dict, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(retro_routes) for i in range(4)])

165
6.345454545454546
n-gram rxn ratio for n in [2, 3, 4, 5]: [0.05952195952195952, 0.032971594615430234, 0.021466973886328723, 0.016336633663366337]
n-gram template ratio for n in [2, 3, 4, 5]: [0.3122605340787159, 0.16927809633289087, 0.14930060978448076, 0.09152915291529153]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.8848484848484849, 0.7515151515151515, 0.6121212121212121]


In [38]:
retro_routes = json.load(open('test_routes/routes_retrostarplus.json'))['routes']
retro_routes = [[rxn.split('>')[0] + '>>' + rxn.split('>')[-1] for rxn in route.split('|')] for route in retro_routes if isinstance(route, str)]
print(len(retro_routes))
print(np.mean([len(route) for route in retro_routes]))
bleu_ratio, bleu_template_ratio = evaluate_routes(retro_routes, test_dict, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(retro_routes) for i in range(4)])

183
6.819672131147541
n-gram rxn ratio for n in [2, 3, 4, 5]: [0.03629476170459777, 0.01557709358914178, 0.008789954337899543, 0.005601851851851853]
n-gram template ratio for n in [2, 3, 4, 5]: [0.2948249761774352, 0.13622888156020685, 0.10181610626816105, 0.06191498316498316]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.907103825136612, 0.7978142076502732, 0.6557377049180327]


In [39]:
retro_routes = json.load(open('test_routes/routes_egmcts.json'))['routes']
retro_routes = [[rxn.split('>')[0] + '>>' + rxn.split('>')[-1] for rxn in route.split('|')] for route in retro_routes if isinstance(route, str)]
print(len(retro_routes))
print(np.mean([len(route) for route in retro_routes]))
bleu_ratio, bleu_template_ratio = evaluate_routes(retro_routes, test_dict, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(retro_routes) for i in range(4)])


183
5.693989071038251
n-gram rxn ratio for n in [2, 3, 4, 5]: [0.012043936634100567, 0.005303030303030303, 0.0010741138560687433, 0.0]
n-gram template ratio for n in [2, 3, 4, 5]: [0.13943001443001443, 0.05208754208754209, 0.030612244897959183, 0.018691588785046728]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.9016393442622951, 0.726775956284153, 0.5846994535519126]


In [40]:
retro_routes = json.load(open('test_routes/routes_retrograph.json'))
print(len(retro_routes))
print(np.mean([len(route) for route in retro_routes]))
bleu_ratio, bleu_template_ratio = evaluate_routes(retro_routes, test_dict, set_ngram_templates, set_ngram_rxns)
print(f'n-gram rxn ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_ratio[i]) for i in range(4)])
print(f'n-gram template ratio for n in [2, 3, 4, 5]:', [np.mean(bleu_template_ratio[i]) for i in range(4)])
print(f'n-gram existence ratio for n in [2, 3, 4, 5]:', [len(bleu_ratio[i]) / len(retro_routes) for i in range(4)])

189
6.402116402116402
n-gram rxn ratio for n in [2, 3, 4, 5]: [0.02071050642479214, 0.009454191033138402, 0.0028169014084507044, 0.0022727272727272726]
n-gram template ratio for n in [2, 3, 4, 5]: [0.20086029490791393, 0.07621598674230252, 0.04459026888604353, 0.01956382001836547]
n-gram existence ratio for n in [2, 3, 4, 5]: [1.0, 0.9047619047619048, 0.7513227513227513, 0.582010582010582]
